In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import joblib
import time
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
from torch.amp import autocast, GradScaler
from torch.utils.data import TensorDataset, DataLoader

In [2]:
# Enable cuDNN benchmarking for faster training
torch.backends.cudnn.benchmark = True

In [3]:
# 1. Load dataset and remove rows with '-' in the tag column
df = pd.read_csv("figma_dataset.csv")

df = df[~df['tag'].str.contains(r'-', regex=True)]
df = df[~df['tag'].str.contains(r'\b(CNX|ADDRESS|ASIDE|CANVAS|CITE|DD|DL|DT|ICON|S|VECTOR|DEL|LEGEND|BDI|LOGO|OBJECT|OPTGROUP|CENTER|CODE|BLOCKQUOTE|FRONT|Q|IFRAME|A|HR|SEARCH|DETAILS|FIELDSET|SLOT|SVG)\b', regex=True)]

# Define the regex pattern for matching
pattern = r'-|\b(CNX|ADDRESS|ASIDE|CANVAS|CITE|DD|DL|DT|ICON|S|VECTOR|DEL|LEGEND|BDI|LOGO|OBJECT|OPTGROUP|CENTER|CODE|BLOCKQUOTE|FRONT|Q|IFRAME|SEARCH|DETAILS|FIELDSET|SLOT)\b'

# Apply the replacement conditionally
for col in ['prev_sibling_html_tag', 'child_1_html_tag', 'child_2_html_tag']:
    df[col] = np.where(df[col].str.contains(pattern, regex=True, na=False), 'DIV', df[col])

# Define mapping for tag replacements
tag_mapping = {
    "ARTICLE": "DIV", "DIV": "DIV", "FIGURE": "DIV", "FOOTER": "DIV", "HEADER": "DIV", "NAV": "DIV", "MAIN": "DIV",
    "BODY" : "DIV", "FORM" : "DIV", "OL" : "DIV", "UL" : "DIV", "TABLE": "DIV", "THEAD":"DIV" , "TBODY": "DIV", "SECTION" : "DIV",
    "H1": "P", "H2": "P", "H3": "P", "H4": "P", "H5": "P", "H6": "P","SUP": "P",
    "P": "P", "CAPTION": "P", "FIGCAPTION": "P", "B": "P", "EM": "P", "I": "P", "TD": "P", "TH": "P", "TR": "P","PRE":"P",
    "U": "P", "TIME": "P", "TXT": "P", "ABBR": "P","SMALL": "P","STRONG": "P","SUMMARY": "P","SPAN": "P", "LABEL": "P","LI":"P",
    "PICTURE": "IMG" , "VIDEO": "IMG",
    "SELECT": "INPUT","TEXTAREA": "INPUT",
    "VECTOR": "SVG"
}

# df.loc[(df["tag"] == "LABEL") & ((df["type"] == "RECTANGLE") | (df["type"] == "GROUP")), "tag"] = "DIV"
df.loc[(df["tag"] == "SPAN") & ((df["type"] == "RECTANGLE") | (df["type"] == "GROUP")), "tag"] = "DIV"

# Replace any value in children tag columns that contains '-' with 'DIV'
children_cols = ['child_1_html_tag', 'child_2_html_tag']
for col in children_cols:
    df[col] = df[col].apply(lambda x: "DIV" if isinstance(x, str) and '-' in x else x)

# Convert tag and parent_tag_html columns to uppercase
df['tag'] = df['tag'].str.upper()
df['prev_sibling_html_tag'] = df['prev_sibling_html_tag'].str.upper()
df['child_1_html_tag'] = df['child_1_html_tag'].str.upper()
df['child_2_html_tag'] = df['child_2_html_tag'].str.upper()

# Apply mapping to 'tag' and 'parent_tag_html' columns
df['tag'] = df['tag'].replace(tag_mapping)
df['prev_sibling_html_tag'] = df['prev_sibling_html_tag'].replace(tag_mapping)
df['child_1_html_tag'] = df['child_1_html_tag'].replace(tag_mapping)
df['child_2_html_tag'] = df['child_2_html_tag'].replace(tag_mapping)

df = df[~df['tag'].str.contains(r'\b(P|IMG)\b', regex=True)]

C:\Users\AOZ\AppData\Local\Temp\ipykernel_3748\2337983856.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df['tag'].str.contains(r'\b(CNX|ADDRESS|ASIDE|CANVAS|CITE|DD|DL|DT|ICON|S|VECTOR|DEL|LEGEND|BDI|LOGO|OBJECT|OPTGROUP|CENTER|CODE|BLOCKQUOTE|FRONT|Q|IFRAME|A|HR|SEARCH|DETAILS|FIELDSET|SLOT|SVG)\b', regex=True)]
C:\Users\AOZ\AppData\Local\Temp\ipykernel_3748\2337983856.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[col] = np.where(df[col].str.contains(pattern, regex=True, na=False), 'DIV', df[col])
C:\Users\AOZ\AppData\Local\Temp\ipykernel_3748\2337983856.py:46: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df['tag'].str.contains(r'\b(P|IMG)\b', regex=True)]


In [4]:
# 2. Separate features and target
y = df["tag"]
X = df.drop(columns=["tag"])

In [5]:
# 3. Identify categorical and continuous columns
categorical_cols = ['type','prev_sibling_html_tag','child_1_html_tag','child_2_html_tag']
continuous_cols = [col for col in X.columns if col not in categorical_cols and col != 'nearest_text_semantic']

# Handle nearest_text_semantic column safely
if 'nearest_text_semantic' in X.columns and X['nearest_text_semantic'].notna().all():
    X['nearest_text_semantic'] = X['nearest_text_semantic'].apply(eval)  # Ensure list format
    embedding_dim = len(X['nearest_text_semantic'].iloc[0])
    nearest_text_semantic_expanded = np.vstack(X['nearest_text_semantic'].values)
else:
    embedding_dim = 384
    nearest_text_semantic_expanded = np.zeros((len(X), embedding_dim))  # Correct dimension for embeddings

# Process categorical features with OneHotEncoder instead of LabelEncoder
X[categorical_cols] = X[categorical_cols].astype(str).fillna('unknown')
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_cat_encoded = ohe.fit_transform(X[categorical_cols])
joblib.dump(ohe, "ohe_encoder.pkl")

# Better missing value handling with imputer
imputer = SimpleImputer(strategy='mean')  # Changed to mean for numerical features
X_continuous_imputed = imputer.fit_transform(X[continuous_cols])
joblib.dump(imputer, "imputer.pkl")

# Scale continuous features
scaler = StandardScaler()
X_continuous_scaled = scaler.fit_transform(X_continuous_imputed)
joblib.dump(scaler, "scaler.pkl")

# Normalize embeddings separately to maintain consistency
scaler_emb = StandardScaler()
nearest_text_semantic_scaled = scaler_emb.fit_transform(nearest_text_semantic_expanded) if embedding_dim > 0 else nearest_text_semantic_expanded
joblib.dump(scaler_emb, "scaler_emb.pkl")

# Get dimensionality information for model architecture
cat_dim = X_cat_encoded.shape[1]
cont_dim = X_continuous_scaled.shape[1]
emb_dim = nearest_text_semantic_scaled.shape[1]

print(f"Feature dimensions - Categorical: {cat_dim}, Continuous: {cont_dim}, Embedding: {emb_dim}")

# Combine one-hot encoded categorical features with scaled continuous features
X_processed = np.concatenate([X_cat_encoded, X_continuous_scaled, nearest_text_semantic_scaled], axis=1)

Feature dimensions - Categorical: 33, Continuous: 13, Embedding: 384


In [6]:
# 4. Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
joblib.dump(label_encoder, "label_encoder.pkl")

from collections import Counter

# Count occurrences of each class
class_counts = Counter(y_encoded)
print(f"Class distribution: {class_counts}")

# Find classes with only 1 sample
rare_classes = [cls for cls, count in class_counts.items() if count < 2]

# Duplicate rare class samples
for cls in rare_classes:
    idx = np.where(y_encoded == cls)[0][0]  # Get the index of the rare sample
    original_class_name = label_encoder.inverse_transform([cls])[0]  # Convert back to original label
    print(f"Duplicating class '{original_class_name}' (only 1 sample present).")

    X_processed = np.vstack([X_processed, X_processed[idx]])  # Duplicate features
    y_encoded = np.append(y_encoded, y_encoded[idx])  # Duplicate label

Class distribution: Counter({2: 595664, 0: 24662, 3: 1815, 1: 2, 4: 2})


In [7]:
# 5. Train/test split - remove stratification if there are classes with too few samples
unique_counts = np.unique(y_encoded, return_counts=True)
min_samples = min(unique_counts[1])

if min_samples < 2:
    print(f"Warning: The least populated class has only {min_samples} sample(s). Removing stratification.")
    X_train, X_test, y_train, y_test = train_test_split(
        X_processed, y_encoded, test_size=0.2, random_state=42
    )
else:
    X_train, X_test, y_train, y_test = train_test_split(
        X_processed, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )

# Move GPU setup earlier
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Convert data to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create dataset and dataloaders with more efficient settings
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(
    train_dataset, 
    batch_size=512,  # Larger batch size
    shuffle=True, 
    num_workers=8,   # Parallel loading
    pin_memory=True,  # Faster data transfer to GPU
    prefetch_factor=2
)

# Compute class weights
print("Computing class weights...")
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

Using device: cuda
Computing class weights...


In [ ]:
# 6. Define improved model architecture with separate pathways for different feature types
class ImprovedTagClassifier(nn.Module):
    def __init__(self, cat_dim, cont_dim, emb_dim, output_size, dropout_rate=0.3):
        super(ImprovedTagClassifier, self).__init__()
        # Embeddings pathway (special handling for semantic embeddings)
        self.emb_fc = nn.Linear(emb_dim, 128)
        self.emb_bn = nn.BatchNorm1d(128)
        self.emb_dropout = nn.Dropout(dropout_rate * 0.5)  # Lower dropout for embeddings
        
        # Categorical features pathway
        self.cat_fc = nn.Linear(cat_dim, 128)
        self.cat_bn = nn.BatchNorm1d(128)
        
        # Continuous features pathway
        self.cont_fc = nn.Linear(cont_dim, 64)
        self.cont_bn = nn.BatchNorm1d(64)
        
        # Combined pathway
        self.combined_dim = 128 + 128 + 64  # Combined dimensions
        
        # Main network after feature combination
        self.fc1 = nn.Linear(self.combined_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, output_size)
        
        self.dropout = nn.Dropout(dropout_rate)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # Split input into its component parts
        cat_features = x[:, :cat_dim]
        cont_features = x[:, cat_dim:cat_dim+cont_dim]
        emb_features = x[:, cat_dim+cont_dim:]
        
        # Process embeddings pathway
        emb = self.relu(self.emb_bn(self.emb_fc(emb_features)))
        emb = self.emb_dropout(emb)
        
        # Process categorical features
        cat = self.relu(self.cat_bn(self.cat_fc(cat_features)))
        cat = self.dropout(cat)
        
        # Process continuous features
        cont = self.relu(self.cont_bn(self.cont_fc(cont_features)))
        cont = self.dropout(cont)
        
        # Combine all features
        combined = torch.cat((emb, cat, cont), dim=1)
        
        # Main network
        x = self.dropout(self.relu(self.bn1(self.fc1(combined))))
        x = self.dropout(self.relu(self.bn2(self.fc2(x))))
        logits = self.fc3(x)
        
        return logits

# Initialize model with correct dimensions
print("Initializing model...")
input_size = X_train.shape[1]
output_size = len(label_encoder.classes_)
print(f"Total input size: {input_size}, Output size (classes): {output_size}")
model = ImprovedTagClassifier(cat_dim, cont_dim, emb_dim, output_size).to(device)

# Print model summary
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model created with {total_params:,} trainable parameters")

Initializing model...
Total input size: 430, Output size (classes): 5
Model created with 171,653 trainable parameters


In [9]:
# 7. Define loss function and optimizer with improved learning rate and weight decay
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

# Improved learning rate scheduler with warmup
from torch.optim.lr_scheduler import OneCycleLR

# Define number of steps
steps_per_epoch = len(train_loader)
total_steps = steps_per_epoch * 200  # 200 epochs max

# Use OneCycleLR for better convergence
scheduler = OneCycleLR(
    optimizer,
    max_lr=0.005,
    total_steps=total_steps,
    pct_start=0.1,  # Use 10% of iterations for warmup
    div_factor=25,  # Initial lr = max_lr/div_factor
    final_div_factor=1000,  # Final lr = max_lr/final_div_factor
)

In [10]:
# 8. Setup mixed precision training
scaler = GradScaler(device='cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
# 9. Training loop with timing and early stopping
print("Starting training...")
best_loss = float('inf')
patience = 15  # Increased patience
counter = 0
early_stop = False
start_time = time.time()

num_epochs = 200
for epoch in range(num_epochs):
    epoch_start = time.time()
    model.train()
    epoch_loss = 0
    
    for batch_X, batch_y in train_loader:
        # Move batch to device
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        
        # Use mixed precision for faster training
        with torch.amp.autocast('cuda', enabled=device.type=='cuda'):
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
        
        # Scale gradients and optimize
        scaler.scale(loss).backward()
        
        # Add gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    
    epoch_time = time.time() - epoch_start
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}, Time: {epoch_time:.2f}s")
    
    # Early stopping with validation evaluation
    if epoch % 3 == 0:  # Validate every 3 epochs to save time
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for batch_X, batch_y in DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=256):
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                outputs = model(batch_X)
                val_batch_loss = criterion(outputs, batch_y)
                val_loss += val_batch_loss.item()
                
                _, predicted = torch.max(outputs.data, 1)
                total += batch_y.size(0)
                correct += (predicted == batch_y).sum().item()
        
        val_avg_loss = val_loss / len(DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=256))
        val_accuracy = correct / total
        
        if (epoch + 1) % 5 == 0:
            print(f"Validation Loss: {val_avg_loss:.4f}, Accuracy: {val_accuracy:.4f}")
        
        # Save based on validation loss instead of training loss
        if val_avg_loss < best_loss:
            best_loss = val_avg_loss
            counter = 0
            print(f"Saving best model with validation loss: {val_avg_loss:.4f}")
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_avg_loss,
                'accuracy': val_accuracy
            }, "best_tag_classifier.pth")
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                early_stop = True
    
    if early_stop:
        break

# Save the final model
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'final_loss': avg_loss
}, "final_tag_classifier.pth")

total_time = time.time() - start_time
print(f"Total training time: {total_time:.2f} seconds")

Starting training...
Saving best model with validation loss: 0.0567
Epoch [5/200], Loss: 0.1406, Time: 8.72s
Epoch [10/200], Loss: 0.1063, Time: 8.23s
Validation Loss: 0.0671, Accuracy: 0.9913
Epoch [15/200], Loss: 0.1320, Time: 8.27s
Epoch [20/200], Loss: 0.0858, Time: 9.50s
Epoch [25/200], Loss: 0.0808, Time: 9.04s
Validation Loss: 0.0711, Accuracy: 0.9971
Epoch [30/200], Loss: 0.0683, Time: 8.54s
Epoch [35/200], Loss: 0.0870, Time: 10.08s
Epoch [40/200], Loss: 0.1119, Time: 9.27s
Validation Loss: 0.2726, Accuracy: 0.9982
Epoch [45/200], Loss: 0.0531, Time: 9.19s
Early stopping at epoch 46
Total training time: 597.87 seconds


In [13]:
# 10. Evaluation on the test set
print("Evaluating model...")
# Load best model
checkpoint = torch.load("best_tag_classifier.pth")
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Process test data in batches for memory efficiency
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=256)

all_predictions = []
all_labels = []
all_probabilities = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model(batch_X)
        probabilities = torch.softmax(outputs, dim=1)
        _, predictions = torch.max(outputs, 1)
        
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())
        all_probabilities.append(probabilities.cpu().numpy())

y_pred = np.array(all_predictions)
y_test_np = np.array(all_labels)
all_probabilities = np.vstack(all_probabilities)

accuracy = accuracy_score(y_test_np, y_pred)
print(f"\nAccuracy: {accuracy:.4f}")

print("\nClassification Report:")
report = classification_report(
    y_test_np, 
    y_pred,
    labels=np.unique(y_test_np),
    target_names=label_encoder.inverse_transform(np.unique(y_test_np)),
    output_dict=True
)

# Convert to DataFrame for better formatting
report_df = pd.DataFrame(report).transpose()
print(report_df)

# Calculate confidence metrics
confidence_scores = np.max(all_probabilities, axis=1)
mean_confidence = np.mean(confidence_scores)
mean_confidence_correct = np.mean(confidence_scores[y_pred == y_test_np])
mean_confidence_incorrect = np.mean(confidence_scores[y_pred != y_test_np]) if np.any(y_pred != y_test_np) else 0

print(f"\nModel Confidence Analysis:")
print(f"Mean prediction confidence: {mean_confidence:.4f}")
print(f"Mean confidence for correct predictions: {mean_confidence_correct:.4f}")
print(f"Mean confidence for incorrect predictions: {mean_confidence_incorrect:.4f}")

# Save model architecture and parameters info for future reference
model_info = {
    'input_size': input_size,
    'categorical_dim': cat_dim,
    'continuous_dim': cont_dim,
    'embedding_dim': emb_dim,
    'output_size': output_size,
    'model_params': total_params,
    'accuracy': accuracy,
    'best_val_loss': best_loss
}

import json
with open('model_info.json', 'w') as f:
    json.dump(model_info, f)

print("\nModel information saved to model_info.json")

# Feature importance analysis
print("\nAnalyzing feature importance...")
# Combine importance from all pathways
importances = {}

# Process embedding pathway
with torch.no_grad():
    emb_weights = model.emb_fc.weight.cpu().numpy()
    emb_importance = np.abs(emb_weights).mean(axis=0)
    
    # Process categorical pathway
    cat_weights = model.cat_fc.weight.cpu().numpy()
    cat_importance = np.abs(cat_weights).mean(axis=0)
    
    # Process continuous pathway
    cont_weights = model.cont_fc.weight.cpu().numpy()
    cont_importance = np.abs(cont_weights).mean(axis=0)
    
    # Get categorical feature names (from one-hot encoder)
    cat_feature_names = ohe.get_feature_names_out(categorical_cols)
    
    # Map weights to feature names
    for i, feat in enumerate(cat_feature_names):
        importances[feat] = float(cat_importance[i])
    
    for i, feat in enumerate(continuous_cols):
        importances[feat] = float(cont_importance[i])
    
    # For embeddings, create generic names
    for i in range(emb_dim):
        importances[f'embedding_{i}'] = float(emb_importance[i])
    
    # Sort by importance
    sorted_importances = sorted(importances.items(), key=lambda x: x[1], reverse=True)
    
    print("\nTop 50 most important features:")
    for feature, imp in sorted_importances[:50]:
        print(f"{feature}: {imp:.4f}")

# Save feature importances
with open('feature_importances.json', 'w') as f:
    json.dump(dict(sorted_importances), f)

print("\nFeature importances saved to feature_importances.json")

# Function to make predictions on new data
def predict_tag(new_data, model, label_encoder, ohe, imputer, scaler, scaler_emb):
    # Prepare the data
    categorical_data = new_data[categorical_cols].astype(str).fillna('unknown')
    continuous_data = new_data[continuous_cols]
    
    # Process categorical features
    cat_encoded = ohe.transform(categorical_data)
    
    # Process continuous features
    cont_imputed = imputer.transform(continuous_data)
    cont_scaled = scaler.transform(cont_imputed)
    
    # Process embeddings
    if 'nearest_text_semantic' in new_data.columns:
        emb_data = np.vstack(new_data['nearest_text_semantic'].apply(eval).values)
        emb_scaled = scaler_emb.transform(emb_data)
    else:
        emb_scaled = np.zeros((len(new_data), embedding_dim))
    
    # Combine features
    X_processed = np.concatenate([cat_encoded, cont_scaled, emb_scaled], axis=1)
    
    # Convert to tensor
    X_tensor = torch.tensor(X_processed, dtype=torch.float32).to(device)
    
    # Make predictions
    model.eval()
    with torch.no_grad():
        outputs = model(X_tensor)
        probabilities = torch.softmax(outputs, dim=1)
        _, predictions = torch.max(outputs, 1)
    
    # Convert to original labels
    predicted_tags = label_encoder.inverse_transform(predictions.cpu().numpy())
    prediction_probs = probabilities.cpu().numpy()
    
    return predicted_tags, prediction_probs

# Save the prediction function
import dill
with open('predict_function.pkl', 'wb') as f:
    dill.dump(predict_tag, f)

print("\nPrediction function saved to predict_function.pkl")
print("Model training and evaluation complete!")

Evaluating model...


C:\Users\AOZ\AppData\Local\Temp\ipykernel_3748\1841807080.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("best_tag_classifier.pth")



Accuracy: 0.9813

Classification Report:
              precision    recall  f1-score        support
BUTTON         0.706566  0.986013  0.823221    4933.000000
DIV            0.999478  0.981080  0.990194  119133.000000
INPUT          0.591736  0.986226  0.739669     363.000000
accuracy       0.981291  0.981291  0.981291       0.981291
macro avg      0.765927  0.984439  0.851028  124429.000000
weighted avg   0.986676  0.981291  0.982843  124429.000000

Model Confidence Analysis:
Mean prediction confidence: 0.9895
Mean confidence for correct predictions: 0.9928
Mean confidence for incorrect predictions: 0.8167

Model information saved to model_info.json

Analyzing feature importance...

Top 50 most important features:
width: 0.1570
border_radius: 0.1554
aspect_ratio: 0.1500
sibling_count: 0.1453
has_background_color: 0.1452
child_1_percentage_of_parent: 0.1438
chars_count_to_end: 0.1358
height: 0.1343
num_children_to_end: 0.1332
child_2_percentage_of_parent: 0.1328
text_length: 0.1237
ne